In [1]:
# Necessary imports

import numpy as np
import cv2
import os
import glob
import pickle
import time
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm

from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from skimage.feature import hog

from helper_functions import *

In [2]:
def extract_features(imgs):
    color_space = 'HSV' # Can be RGB, HSV, LAB, HLS, YUV, YCrCb
    orient = 12  # HOG orientations
    pix_per_cell = 8 # HOG pixels per cell
    cell_per_block = 2 # HOG cells per block
    hog_channel = 'ALL' # Can be 0, 1, 2, or "ALL"
    spatial_size = (24, 24) # Spatial binning dimensions
    hist_bins = 24    # Number of histogram bins
    spatial_feat = True # Spatial features on or off
    hist_feat = True # Histogram features on or off
    hog_feat = True # HOG features on or off
    
    features = []
    for img_path in tqdm(imgs):
        img = mpimg.imread(img_path)
        
        features.append(single_img_features(
            img,
            color_space=color_space,
            spatial_size=spatial_size,
            hist_bins=hist_bins,
            orient=orient,
            pix_per_cell=pix_per_cell,
            cell_per_block=cell_per_block,
            hog_channel=hog_channel,
            spatial_feat=spatial_feat,
            hist_feat=hist_feat,
            hog_feat=hog_feat
        ))
    
    return features

In [3]:
### Feature Extraction

# Load pickled raw data set
with open('rawdata.p', mode='rb') as f:
    data = pickle.load(f)
    
cars_train = data['cars_train']
notcars_train = data['notcars_train']

cars_valid = data['cars_valid']
notcars_valid = data['notcars_valid']

cars_test = data['cars_test']
notcars_test = data['notcars_test']

# Extract features
print('Extracting features...')
start = time.time()

cars_features_train = extract_features(cars_train)
notcars_features_train = extract_features(notcars_train)

cars_features_valid = extract_features(cars_valid)
notcars_features_valid = extract_features(notcars_valid)

cars_features_test = extract_features(cars_test)
notcars_features_test = extract_features(notcars_test)

end = time.time()
print('Time taken to extract features (Spatial, Color Hist, HOG): {}'.format(end-start))

Extracting features...


/Users/sid/anaconda/envs/carnd-term1/lib/python3.5/site-packages/skimage/feature/_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)








Time taken to extract features (Spatial, Color Hist, HOG): 123.28962707519531


In [4]:
### Data set exploratory visualisation and summarisation

print('Length of feature vector: {}'.format(len(cars_features_train[0])))

Length of feature vector: 8856


In [5]:
### Pre-processing

X_train = np.vstack([cars_features_train, notcars_features_train])
X_valid = np.vstack([cars_features_valid, notcars_features_valid])
X_test = np.vstack([cars_features_test, notcars_features_test])

y_train = np.hstack([np.ones(len(cars_features_train)), np.zeros(len(notcars_features_train))])
y_valid = np.hstack([np.ones(len(cars_features_valid)), np.zeros(len(notcars_features_valid))])
y_test = np.hstack([np.ones(len(cars_features_test)), np.zeros(len(notcars_features_test))])

scaler = StandardScaler().fit(X_train)

X_train = scaler.transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

# Sanity check

assert(len(X_train) == len(y_train))
assert(len(X_valid) == len(y_valid))
assert(len(X_test) == len(y_test))

In [6]:
### Define and implement model architecture
### Training and evaluation pipeline

clf = LinearSVC()

print('Starting training...')
start = time.time()
clf.fit(X_train, y_train)
end = time.time()
print('Training took: {}'.format(end-start))

Starting training...
Training took: 5.363823890686035


In [7]:
print('Training accuracy: {}'.format(clf.score(X_train, y_train)))
print('Validation accuracy: {}'.format(clf.score(X_valid, y_valid)))

Training accuracy: 1.0
Validation accuracy: 0.990990990990991


In [ ]:
### Testing accuracy